## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-15-14-48-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...
1,2025-07-15-14-44-51 +0000,bbc,Crews battle overnight fire in Kilmarnock town...,https://www.bbc.com/news/videos/cwyq2yxznxyo
2,2025-07-15-14-39-12 +0000,bbc,Immigration status of benefit claimants publis...,https://www.bbc.com/news/articles/cdx5pw8pwg5o
3,2025-07-15-14-29-01 +0000,nyt,China Puts New Restrictions on E.V. Battery Ma...,https://www.nytimes.com/2025/07/15/business/ch...
4,2025-07-15-14-28-45 +0000,nyt,Rubio Visits Asia in Shadow of Trump’s Tariffs,https://www.nytimes.com/video/world/asia/10000...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2303/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,trump,17
31,china,7
52,tariff,7
42,tariffs,5
45,up,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
42,2025-07-15-03-45-00 +0000,wsj,China’s economy showed resilience in a turbule...,https://www.wsj.com/world/china/chinas-economy...,55
45,2025-07-15-02-01-34 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o,35
40,2025-07-15-04-30-50 +0000,bbc,China growth beats expectations as Trump tarif...,https://www.bbc.com/news/articles/c20r461g61xo,34
10,2025-07-15-13-32-31 +0000,nyt,China’s Economy Grows Steadily Despite Trump’s...,https://www.nytimes.com/2025/07/14/business/ch...,33
0,2025-07-15-14-48-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...,33


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
42,55,2025-07-15-03-45-00 +0000,wsj,China’s economy showed resilience in a turbule...,https://www.wsj.com/world/china/chinas-economy...
45,18,2025-07-15-02-01-34 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o
0,16,2025-07-15-14-48-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...
27,16,2025-07-15-09-51-44 +0000,nyt,Air India Crash Findings Prompt Inspections of...,https://www.nytimes.com/2025/07/15/world/asia/...
5,16,2025-07-15-14-24-00 +0000,wsj,"Inflation picked up in June, a potential sign ...",https://www.wsj.com/economy/inflation-hit-2-7-...
24,15,2025-07-15-10-01-45 +0000,bbc,Trump shrugs off giving Parliament speech when...,https://www.bbc.com/news/articles/cx2g1xexe7qo
25,14,2025-07-15-10-01-29 +0000,bbc,Watch: Flash flooding sweeps through New York ...,https://www.bbc.com/news/videos/c0l4259g3jdo
33,13,2025-07-15-09-00-14 +0000,nyt,"Amid Devastating Winter Losses, Another Threat...",https://www.nytimes.com/2025/07/15/science/hon...
64,10,2025-07-14-17-11-33 +0000,bbc,MP charged for accusing Lesotho king of signin...,https://www.bbc.com/news/articles/cp86pznd65ko
65,10,2025-07-14-17-05-04 +0000,bbc,Nine killed and dozens hurt in fire at Massach...,https://www.bbc.com/news/articles/cq6mp9ydl73o


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
